# `Group Project` - Boston 2017 Bike Sharing Analysis

## Introduction
In this project we will work with real world data from "Blue Bikes" which is a bike sharing firm based in Boston.

Our focus lies on data of the year 2017 and we will make a comprehensive analysis, in which we use typical data analysis and machine learning approaches to make use of the data to monitize and optimize the operations of "Blue Bikes". The overall topic is smart mobility systems and how we can use data in impactful ways to address pressing societal issues. In terms of bike sharing the addressed societal issue are the reduce of greenhouse gas emissions, reduce of pollution as health risk for urban population, reduce of (fatal) road accidents and to create a more efficient road transport infrastructur. We will cover the following task:

- **Task 1) Data Collection & Preparation**: 
    * Cleaning of datasets for use in later analysis stages
- **Task 2) Descriptive Analysis**: 
    * Demonstrate temporal demand patterns and seasonality
    * Demonstrate geographical demand patterns 
    * Define Key Performance Indicators which provides overview of current fleet operations
- **Task 3) Predictive Analysis**:
    * Forecast total system-level demand in the next hour

**The authors of this analysis are:**
* Robin Kirch      (7364580)
* Niklas Nesseler  (7367375)
* Lukas Tempfli    (7367097)
* Pascal Zajonz    (7369974)
* Sven Dornbrach   (7364484)
* Moritz Danhausen (7369413)


Hint: Please run the seperated cells from the top to the bottom

### Required Imports

In [5]:
import numpy as np
import pandas as pd
#import etc


# `Task 1) Data Collection & Preparation:`

### Details to Task 1
#TODO
Detailed introduction to task 1 here

In [32]:
raw_data_boston = pd.read_csv("boston_2017.csv", sep=",", index_col="start_time")
#Problematisch bei gleicher Startzeit und ggbf. kompletten Dateipfad angeben notwendig

In [8]:
raw_data_boston

,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type
start_time,,,,,,,
2017-01-01 00:06:58,2017-01-01 00:12:49,67,139,MIT at Mass Ave / Amherst St,Dana Park,644,Subscriber
2017-01-01 00:13:16,2017-01-01 00:28:07,36,10,Boston Public Library - 700 Boylston St.,B.U. Central - 725 Comm. Ave.,230,Subscriber
2017-01-01 00:16:17,2017-01-01 00:44:10,36,9,Boston Public Library - 700 Boylston St.,Agganis Arena - 925 Comm Ave.,980,Customer
2017-01-01 00:21:22,2017-01-01 00:33:50,46,19,Christian Science Plaza,Buswell St. at Park Dr.,1834,Subscriber
2017-01-01 00:30:06,2017-01-01 00:40:28,10,8,B.U. Central - 725 Comm. Ave.,Union Square - Brighton Ave. at Cambridge St.,230,Subscriber
...,...,...,...,...,...,...,...
2017-12-31 23:46:18,2017-12-31 23:50:27,117,141,Binney St / Sixth St,Kendall Street,1846,Subscriber
2017-12-29 16:11:56,2017-12-29 16:16:18,54,42,Tremont St at West St,Boylston St at Arlington St TEMPORARY WINTER L...,2,Subscriber
2017-12-30 08:09:44,2017-12-30 08:26:08,54,58,Tremont St at West St,Beacon St at Arlington St,1534,Subscriber


In [12]:
raw_data_boston.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1313774 entries, 2017-01-01 00:06:58 to 2017-12-30 18:27:39
Data columns (total 7 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   end_time            1313774 non-null  object
 1   start_station_id    1313774 non-null  int64 
 2   end_station_id      1313774 non-null  int64 
 3   start_station_name  1313774 non-null  object
 4   end_station_name    1313774 non-null  object
 5   bike_id             1313774 non-null  int64 
 6   user_type           1313774 non-null  object
dtypes: int64(3), object(4)
memory usage: 80.2+ MB


In [27]:
raw_data_boston.describe()

,start_station_id,end_station_id,bike_id
count,1.313774e+06,1.313774e+06,1.313774e+06
mean,8.571053e+01,8.554360e+01,9.931371e+02
std,5.634766e+01,5.648317e+01,5.701397e+02
min,1.000000e+00,1.000000e+00,1.000000e+00
25%,4.300000e+01,4.300000e+01,5.160000e+02
50%,7.400000e+01,7.400000e+01,9.780000e+02
75%,1.170000e+02,1.170000e+02,1.512000e+03
max,2.320000e+02,2.320000e+02,1.981000e+03


In [28]:
raw_data_boston.columns

Index(['start_time', 'start_station_id', 'end_station_id',
       'start_station_name', 'end_station_name', 'bike_id', 'user_type'],
      dtype='object')

In [33]:
raw_data_boston["end_time"].describe()
#start_time nicht zugreifbar

count                 1313774
unique                1245112
top       2017-08-15 17:37:49
freq                        6
Name: end_time, dtype: object

In [18]:
raw_data_boston["start_station_id"].describe()

count    1.313774e+06
mean     8.571053e+01
std      5.634766e+01
min      1.000000e+00
25%      4.300000e+01
50%      7.400000e+01
75%      1.170000e+02
max      2.320000e+02
Name: start_station_id, dtype: float64

In [19]:
raw_data_boston["end_station_id"].describe()

count    1.313774e+06
mean     8.554360e+01
std      5.648317e+01
min      1.000000e+00
25%      4.300000e+01
50%      7.400000e+01
75%      1.170000e+02
max      2.320000e+02
Name: end_station_id, dtype: float64

In [20]:
raw_data_boston["start_station_name"].describe()

count                          1313774
unique                             285
top       MIT at Mass Ave / Amherst St
freq                             42320
Name: start_station_name, dtype: object

In [21]:
raw_data_boston["end_station_name"].describe()

count                          1313774
unique                             283
top       MIT at Mass Ave / Amherst St
freq                             42442
Name: end_station_name, dtype: object

In [22]:
raw_data_boston["bike_id"].describe()

count    1.313774e+06
mean     9.931371e+02
std      5.701397e+02
min      1.000000e+00
25%      5.160000e+02
50%      9.780000e+02
75%      1.512000e+03
max      1.981000e+03
Name: bike_id, dtype: float64

In [23]:
raw_data_boston["user_type"].describe()

count        1313774
unique             2
top       Subscriber
freq         1104738
Name: user_type, dtype: object

In [30]:
raw_data_boston.count()

start_time            1313774
start_station_id      1313774
end_station_id        1313774
start_station_name    1313774
end_station_name      1313774
bike_id               1313774
user_type             1313774
dtype: int64

# `Task 2) Descriptive Analysis:`

### Details to Task 2
#TODO
Detailed introduction to task 2 here

# `Task 3) Descriptive Analysis:`

### Details to Task 3
#TODO
Detailed introduction to task 3 here